In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
plt.rcParams["figure.figsize"] = (12,8)

import warnings
warnings.filterwarnings("ignore")

import datetime as dt

from mftool import Mftool

from nsepy import get_history

import plotly.express as px
import ipywidgets as widgets

from pyxirr import xirr

In [2]:
comp_df = pd.read_csv('USE FOR TEST - comp_df.csv')
comp_df.head()

,date,scheme_code,nav,fund_house,scheme_type,scheme_category,outlier,nifty_value
0,2007-09-18,105460,11.7100,Invesco Mutual Fund,Open Ended Schemes,Equity Scheme - Contra Fund,0,4546.200195
1,2007-09-18,100550,38.2785,Franklin Templeton Mutual Fund,Open Ended Schemes,Hybrid Scheme - Aggressive Hybrid Fund,0,4546.200195
2,2007-09-18,100082,26.6150,DSP Mutual Fund,Open Ended Schemes,Hybrid Scheme - Aggressive Hybrid Fund,0,4546.200195
3,2007-09-18,100081,45.8730,DSP Mutual Fund,Open Ended Schemes,Hybrid Scheme - Aggressive Hybrid Fund,0,4546.200195
4,2007-09-18,106167,42.4400,Canara Robeco Mutual Fund,Open Ended Schemes,Hybrid Scheme - Aggressive Hybrid Fund,0,4546.200195


In [3]:
summary_schemes_df = pd.read_csv('USE FOR TEST - summary_schemes_df.csv')
summary_schemes_df.head()

,scheme_code,mindate,maxdate,min_div_date,max_div_date,file
0,100033,2007-09-18,2022-09-08,2009-09-18,2020-09-08,1
1,100034,2007-09-18,2022-09-08,2009-09-18,2020-09-08,2
2,100063,2007-09-18,2022-09-08,2009-09-18,2020-09-08,3
3,100064,2007-09-18,2022-09-08,2009-09-18,2020-09-08,4
4,100067,2007-09-18,2022-09-08,2009-09-18,2020-09-08,5


In [4]:
comp_df['date'] = pd.to_datetime(comp_df['date'])

In [5]:
summary_schemes_df['mindate'] = pd.to_datetime(summary_schemes_df['mindate'])
summary_schemes_df['maxdate'] = pd.to_datetime(summary_schemes_df['maxdate'])
summary_schemes_df['min_div_date'] = pd.to_datetime(summary_schemes_df['min_div_date'])
summary_schemes_df['max_div_date'] = pd.to_datetime(summary_schemes_df['max_div_date'])

In [6]:
summary_schemes_df['scheme_code'].mean()

117160.68392204629

# Running Test

In [15]:
def calc_nav_nifty_beat(xdffunc, dayfunc):
    
    xdffunc['nifty_units'] = np.where(xdffunc['date'].dt.day==dayfunc, 10000/xdffunc['nifty_value'], 0)
    xdffunc['nav_units'] = np.where(xdffunc['date'].dt.day==dayfunc, 10000/xdffunc['nav'], 0)
    
    datesfunc = xdffunc[xdffunc['date'].dt.day==dayfunc]['date'].reset_index(drop=True)
    datesfunc[len(datesfunc)]=max(xdffunc['date'])
    
    amounts_nav=[-10000]*len(datesfunc)
    amounts_nav[-1] = sum(xdffunc['nav_units'])*xdffunc[xdffunc['date']==max(xdffunc['date'])]['nav']
    
    nav_xirr = xirr(datesfunc, amounts_nav)
    
    amounts_nifty=[-10000]*len(datesfunc)
    amounts_nifty[-1] = sum(xdffunc['nifty_units'])*xdffunc[xdffunc['date']==max(xdffunc['date'])]['nifty_value']
    
    nifty_xirr = xirr(datesfunc, amounts_nifty)
    
    return nav_xirr, nifty_xirr

In [16]:
codes = comp_df['scheme_code'].unique()

In [17]:
done_df = pd.read_csv('results_mf_beat.csv')
done_df.head()

,scheme_code,pre_nav_xirr,post_nav_xirr,pre_nifty_xirr,post_nifty_xirr,div_date,day,pre_beat,post_beat
0,105460,0.184498677,0.154001891,0.127228623,0.128740879,7/24/2014,1,0.057270055,0.025261012
1,105460,0.152225718,0.233901241,0.101878898,0.20612118,1/25/2020,1,0.05034682,0.027780061
2,105460,0.176821461,0.167297399,0.103806361,0.147995143,7/6/2017,1,0.0730151,0.019302256
3,105460,0.084198455,0.167308,0.061737286,0.128106012,4/2/2013,1,0.022461169,0.039201988
4,105460,0.165519767,0.168355483,0.09871424,0.124935671,3/22/2011,1,0.066805527,0.043419812


In [18]:
codes_done=done_df['scheme_code'].unique()
codes_done

array(['105460', '100550', 'scheme_code', '100082', '100081', '106167',
       '106166', '103154', '103155', '103098', '103097', '100549',
       '102428', '103491', 106166, 103154, 103155, 103098, 103490, 103085,
       106168, 100254, 102595, 102594, 101765, 101764, 101672, 100497,
       102948, 100356, 100601, 101816, 101818, 102846, 102847, 102848,
       100685, 100684, 101222, 102947, 100414, 101551, 101069, 101070,
       100321, 100323, 100286, 100221, 100220, 100355, 102885, 100496,
       100794, 100795, 101852, 100415, 101833, 103743, 102142, 100643,
       103034, 103114, 104523, 101766, 101295, 103145, 102823, 100644,
       100645, 101263, 101262, 101264, 102431, 102432, 103146, 105417,
       105418, 102401, 100176, 100177, 102874, 102875, 103007, 103006,
       103360, 103361, 105805, 105804, 102398, 102397, 100739, 100740,
       102395, 102396, 106426, 106425, 100806, 100807, 102402, 102450,
       102433, 102451, 100948, 106370, 102573, 102574, 102136, 102135,
     

In [19]:
len(codes)

1642

In [20]:
codes = codes[~np.isin(codes, codes_done)]

In [21]:
len(codes)

1108

In [ ]:
count_outer=1
for scode in codes:
    
    l_scodes = []
    l_pre_nav_irr = []
    l_pre_nifty_irr = []
    l_post_nav_irr = []
    l_post_nifty_irr = []
    l_dates = []
    l_day = []
    
    
    start_time = dt.datetime.now()
    xdf = comp_df[comp_df['scheme_code']==scode]
    
    for day in list(range(1,28,3)):
        
        
        random_dates_df = summary_schemes_df[summary_schemes_df['scheme_code']==scode][['min_div_date', 'max_div_date']]
        random_dates = pd.DataFrame(pd.date_range(min(random_dates_df['min_div_date']), max(random_dates_df['max_div_date'])))
        random_dates.columns=['date']
        random_dates = random_dates.sample(frac=.2)
        
        
        for rand_date in random_dates['date']:
            
            
            pre_df = xdf[xdf['date']<rand_date].copy()
            post_df = xdf[xdf['date']>=rand_date].copy()
            
            ### Pre calculation
            pre_nav_xirr, pre_nifty_xirr = calc_nav_nifty_beat(pre_df, day)
            pre_beat = pre_nav_xirr - pre_nifty_xirr
            ### Post calculation
            post_nav_xirr, post_nifty_xirr = calc_nav_nifty_beat(post_df, day)
            post_beat = post_nav_xirr - post_nifty_xirr
            
            l_scodes.append(scode)
            l_pre_nav_irr.append(pre_nav_xirr)
            l_post_nav_irr.append(post_nav_xirr)
            l_pre_nifty_irr.append(pre_nifty_xirr)
            l_post_nifty_irr.append(post_nifty_xirr)
            l_day.append(day)
            l_dates.append(rand_date)
    
    resultsdict = {'scheme_code' : l_scodes,
               'pre_nav_xirr' : l_pre_nav_irr,
               'post_nav_xirr' : l_post_nav_irr,
               'pre_nifty_xirr' : l_pre_nifty_irr,
               'post_nifty_xirr' : l_post_nifty_irr,
               'div_date' : l_dates,
               'day' : l_day}
    results_df = pd.DataFrame(resultsdict)
    results_df['pre_beat'] = results_df['pre_nav_xirr'] - results_df['pre_nifty_xirr']
    results_df['post_beat'] = results_df['post_nav_xirr'] - results_df['post_nifty_xirr']
    
    if count_outer == 1:
        results_df.to_csv('results_mf_beat.csv', mode='a', index=False)
    else :
        results_df.to_csv('results_mf_beat.csv', mode='a', header=False, index=False)
    
    end_time = dt.datetime.now()
    timetaken = round((end_time-start_time).seconds/60,2)
    print(scode, round(count_outer/len(codes)*100,2), " done took - ", timetaken, " mins")
    
    count_outer = count_outer+1
    

101592 0.09  done took -  4.62  mins
106169 0.18  done took -  5.87  mins
106170 0.27  done took -  4.78  mins
106316 0.36  done took -  4.28  mins
106317 0.45  done took -  3.83  mins
106822 0.54  done took -  3.6  mins
106823 0.63  done took -  3.3  mins
106871 0.72  done took -  3.55  mins
113097 0.81  done took -  2.27  mins
113099 0.9  done took -  2.48  mins
112923 0.99  done took -  2.4  mins
112931 1.08  done took -  2.55  mins
112932 1.17  done took -  2.32  mins
113098 1.26  done took -  2.38  mins
112925 1.35  done took -  2.47  mins
112926 1.44  done took -  2.33  mins
112924 1.53  done took -  2.18  mins
112927 1.62  done took -  2.15  mins
113134 1.71  done took -  2.28  mins
113142 1.81  done took -  2.12  mins
113143 1.9  done took -  1.98  mins
113141 1.99  done took -  1.98  mins
113153 2.08  done took -  1.98  mins
113221 2.17  done took -  1.98  mins
113222 2.26  done took -  2.03  mins
113064 2.35  done took -  1.95  mins
113065 2.44  done took -  2.03  mins
113361